Machine Translation

In [1]:
!nvidia-smi

Mon Feb 26 18:06:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# installing libraries through pip
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
# installing transformers accelerate through pip
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [4]:
# importing few libraries and classes
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

# own translation system

# language translation system

# from the hugging face we are going to use open source translation model

# data set also i wll take from hugging face

Building Language Translation System
From the HuggingFace, we are going to use Opensourse Translation Model
# dataset also taking from HuggingFace

In [ ]:
# transfer learning

# FineTuning(for pretrained model)

# RAG system(upcoming)

In [ ]:
# Google T5
# we are going to use 'Helsinki-NLP/opus-mt-zh-en'
# FB-M2M-100
# Google cloud Translation API
# Tune AI
# Amazone Comprend
# i want to create from English to Bengali

In [5]:
# model creation through copy from HuggingFace
model='Helsinki-NLP/opus-mt-en-hi'

In [9]:
# loading the dataset from this directory
dataset=load_dataset("cfilt/iitb-english-hindi")

In [11]:
# the whole dataset
dataset
# we will take test data because it will take
# less time to trained
dataset['test']['translation'][0]

{'en': 'A black box in your car?', 'hi': 'आपकी कार में ब्लैक बॉक्स?'}

In [ ]:
# creating object of the AutoTokenizer() class
# getttig a method of the AutoTokenizer() class
# and load the model
tokenizer = AutoTokenizer.from_pretrained(model)

In [17]:
# calling the object by passing arguments
a=tokenizer("there is Mehedi who is expert in Data Science")

In [ ]:
# a['input_ids']
# a['attention_mask']
# dataset['validation']['translation']

In [ ]:
# all dataset for validation and then printing
# the engligh word means iput words
for ex in  dataset["validation"]["translation"]:
  print(ex["en"])

In [ ]:
# all dataset for validation and then printing
# the hindi words means output words
for ex in  dataset["validation"]["translation"]:
  print(ex["hi"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [23]:
# creating a function where we will take the validation data as ex
# assigning inputs and targets as output and there we use list comprehension
max_input_length = 128
max_target_length = 128

source_lang="en"
target_lang="hi"

def process_function(sentence):

  inputs=[ex[source_lang] for ex in sentence["translation"]]
  targets=[ex[target_lang] for ex in sentence["translation"]]

  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)


  model_inputs["labels"]=labels["input_ids"]
  return model_inputs

In [24]:
# now we are mapping the dataset through
# the function that we just created
tokenized_data=dataset.map(process_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [25]:
# here is our model
model

'Helsinki-NLP/opus-mt-en-hi'

In [26]:
# loading pre trained model through calling TensorFlow() class and give the model to load
model=TFAutoModelForSeq2SeqLM.from_pretrained(model)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [27]:
# assigning values for the parameters for
batch_size=32
learning_rate=0.001
weight_decay=0.01
num_train_epochs=1

In [28]:
# creating an object of the DataCollatorForSeq2Seq() class
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [29]:
# calling function prepare_tf_dataset() through model
# this is for trainig the dataset. because our training dataset
# is huge, so we here using the test dataset
train_dataset=model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator

)

In [30]:
# calling function prepare_tf_dataset() through model
# this is for validating the dataset.
validation_dataset=model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,

)

In [31]:
# there are so many optimizer exist but here
# we are using ADAM optimizer
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

In [32]:
# now we are compiling the model ans passing argument as optimizer
model.compile(optimizer=optimizer)

In [33]:
# now we call fit() method of our model and passing the training dataset as 'test' data
# and validation dataset and the epoch size
model.fit(train_dataset,validation_data=validation_dataset,epochs=2)

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
78/78 [==============================] - 172s 784ms/step - loss: 4.6031 - val_loss: 4.8761
Epoch 2/2
78/78 [==============================] - 49s 625ms/step - loss: 3.3561 - val_loss: 4.9485


In [35]:
# saving the model at your development code editor and here we are using google colab
model.save_pretrained("translation_using_tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


# Model Inferencing


In [44]:
# here is my input text
input_text="Thousands of motorists have already taken the black boxes, some of which have GPS monitoring, for a test drive."

In [45]:
# now we call tokenizer with passing input text and return the data as numpy
tokenized=tokenizer([input_text],return_tensors='np')
tokenized

{'input_ids': array([[15385,     8, 19396,  9810,    55,   927,  1233,     4,  3078,
        18743,     2,   241,     8,   157,    55, 22238, 38940,     2,
           34,    19,  2564,  3010,     3,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]])}

In [46]:
# now let the model generate the tokens for the target
out=model.generate(**tokenized, max_length=128)
out[0]

<tf.Tensor: shape=(37,), dtype=int32, numpy=
array([61949,   494, 26619,    37,   204,    12,    58, 20501,  8323,
        4765,    57,  1217,     5,     2,   994,    12,    95,    99,
          18,  7268,     6,    39,  1482,  1877,    12,  2838,    78,
           5,    40,     0, 61949, 61949, 61949, 61949, 61949, 61949,
       61949], dtype=int32)>

In [47]:
# here is showing the output
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

कई कंपनियों ने पहले से ही ब्लैक बॉक्स ग्रहण कर ली है, जिसमें से कुछ लोगों को परीक्षण के लिए निश्चित तरीके से निर्धारित किया है।
